In [6]:
import pandas as pd
import geopandas as gpd
df = pd.read_csv('../output/companies-gaming-highlyfunded-starbucks.csv')
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf.head()

Unnamed: 0                       _id       name total_money_raised  \
0           0  5e41272299bd8e3148da766e      Curse               $12M   
1           2  5e41272299bd8e3148da76bd    Grockit             $44.7M   
2           3  5e41272299bd8e3148da77cf  MocoSpace             $10.5M   
3           4  5e41272299bd8e3148da7816     OMGPOP             $16.6M   
4           6  5e41272299bd8e3148da7863   FlowPlay             $3.97M   

    latitude   longitude                     geometry  
0  37.787092 -122.399972  POINT (-122.39997 37.78709)  
1  37.775196 -122.419204  POINT (-122.41920 37.77520)  
2  42.350274  -71.058768   POINT (-71.05877 42.35027)  
3  40.723384  -74.001704   POINT (-74.00170 40.72338)  
4  47.601532 -122.335943  POINT (-122.33594 47.60153)

In [19]:
import os
from dotenv import load_dotenv
load_dotenv()

client_id = os.getenv("FourSquareID")
client_secret = os.getenv("FourSquareSecret")
print(f"We have a Foursquare id: {client_id[0]} and secret {client_secret[0]}")

Python-dotenv could not parse statement starting at line 4
We have a Foursquare id: N and secret V


# Making functions

After some testing, we can make some functions to get the information we want from the foursquare API. 
Lets make use a function to get the number of daycare centers within 10km of the company location.

In [86]:
import json, requests

def getVenues(lat,long,venuecode,radius=10000):
    url = 'https://api.foursquare.com/v2/venues/search'
    params = dict(
    client_id=client_id,
    client_secret=client_secret,
    v='20180323',
    ll=f'{lat},{long}',
    categoryId='5744ccdfe4b0c0459246b4c7',
    limit=50,
    radius=radius
    )
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    return data

def countVenues(data):
    return len(data['response']['venues'])

def numberOfVenuesNear(dfnames, dflat, dflong, venuecode, radius=10000):
    list = []
    for name, lat, long in zip(dfnames, dflat, dflong):
        list.append([name, lat, long, countVenues(getVenues(lat, long, venuecode, radius))])
    return list


Now that we have made the functions, we can do some testing - and it seems to work great!

In [87]:
venuecode = '5744ccdfe4b0c0459246b4c7' # Venuecode for daycare
comps = numberOfVenuesNear(gdf.name, gdf.latitude, gdf.longitude, venuecode, radius = 5000)

In [92]:
compdaycare = pd.DataFrame(comps, columns = ['name','latitude', 'longitutude','numberofdaycare'])
compdaycare

name   latitude  longitutude  numberofdaycare
0                   Curse  37.787092  -122.399972               42
1                 Grockit  37.775196  -122.419204               44
2               MocoSpace  42.350274   -71.058768               33
3                  OMGPOP  40.723384   -74.001704               43
4                FlowPlay  47.601532  -122.335943               26
5                GamerDNA  42.375392   -71.118487               33
6                PlaySpan  37.396312  -121.961396               13
7               PlayFirst  37.790346  -122.401850               42
8         Challenge Games  30.268236   -97.740651               11
9                Outspark  37.775196  -122.419204               44
10                  Zynga  37.765158  -122.404234               43
11               RocketOn  37.654660  -122.407880               10
12                TwoFish  37.522281  -122.258139               25
13            Three Rings  37.775196  -122.419204               44
14  Social Gaming Network  37.446823  -122.161523               22
15  Social Gaming Network  34.069849  -118.405418                9
16            crowdSPRING  41.885830   -87.657354               41
17              99designs  37.795531  -122.400598               42
18              99designs -37.802659   144.986855               42
19              99designs  52.498620    13.446903               45
20                  Akoha  45.516545   -73.578477               40
21               Playfish  51.499109    -0.198480               23
22               Playfish  69.636216    18.931062                5
23                  Pikum  51.539778    -0.152998               14
24        Realtime Worlds  56.461428    -2.968111                0
25        Realtime Worlds  40.010492  -105.276843                9
26               Cellufun  40.739930   -73.993049               44
27              eRepublik  40.430940    -3.695290               26
28               WeeWorld  42.458900   -71.349521                1
29               WeeWorld  55.863171    -4.263517                3
30                Turbine  37.477652  -122.223384               18
31                Turbine  42.214483   -71.185964                7
32              Sometrics  34.048857  -118.251343               13
33               PurePlay  37.775196  -122.419204               44
34             Riot Games  34.002591  -118.394238               17
35                 Minted  37.797435  -122.403175               42
36                  Smule  37.779624  -122.391236               41
37                GotGame  37.777720  -122.395785               42
38          Double Fusion  37.788714  -122.392714               42
39   Unkasoft Advergaming  40.447436    -3.671492               26
40   Unkasoft Advergaming  41.001663    -5.693793                0
41                  Geewa  50.093849    14.450534               39
42           Crispy Gamer  40.750597   -73.985273               42
43                Owlient  48.830809     2.303220               17
44                 SCVNGR  42.340653   -71.068003               34
45            Virgin Play  40.427308    -3.689512               26
46  NetLogic Microsystems  37.420408  -122.088466               20
47               Bigpoint  53.597305     9.976252               42
48               Altobeam  55.902143    -3.208711                3

In [89]:
compdaycare.to_csv("../output/compsanddaycare.csv")

And with this we have a list of companies, specialized in design/gaming, with at least 1M USD raised in funding, within 2km of a starbucks, and with the number of daycare venues within a 5km radius noted next to it.

Let us sort the result CSV as well.

In [96]:
compdaycare.sort_values(by = ['numberofdaycare'], ascending = False)

name   latitude  longitutude  numberofdaycare
19              99designs  52.498620    13.446903               45
33               PurePlay  37.775196  -122.419204               44
13            Three Rings  37.775196  -122.419204               44
26               Cellufun  40.739930   -73.993049               44
1                 Grockit  37.775196  -122.419204               44
9                Outspark  37.775196  -122.419204               44
3                  OMGPOP  40.723384   -74.001704               43
10                  Zynga  37.765158  -122.404234               43
35                 Minted  37.797435  -122.403175               42
47               Bigpoint  53.597305     9.976252               42
37                GotGame  37.777720  -122.395785               42
38          Double Fusion  37.788714  -122.392714               42
18              99designs -37.802659   144.986855               42
0                   Curse  37.787092  -122.399972               42
7               PlayFirst  37.790346  -122.401850               42
42           Crispy Gamer  40.750597   -73.985273               42
17              99designs  37.795531  -122.400598               42
16            crowdSPRING  41.885830   -87.657354               41
36                  Smule  37.779624  -122.391236               41
20                  Akoha  45.516545   -73.578477               40
41                  Geewa  50.093849    14.450534               39
44                 SCVNGR  42.340653   -71.068003               34
5                GamerDNA  42.375392   -71.118487               33
2               MocoSpace  42.350274   -71.058768               33
45            Virgin Play  40.427308    -3.689512               26
39   Unkasoft Advergaming  40.447436    -3.671492               26
27              eRepublik  40.430940    -3.695290               26
4                FlowPlay  47.601532  -122.335943               26
12                TwoFish  37.522281  -122.258139               25
21               Playfish  51.499109    -0.198480               23
14  Social Gaming Network  37.446823  -122.161523               22
46  NetLogic Microsystems  37.420408  -122.088466               20
30                Turbine  37.477652  -122.223384               18
34             Riot Games  34.002591  -118.394238               17
43                Owlient  48.830809     2.303220               17
23                  Pikum  51.539778    -0.152998               14
32              Sometrics  34.048857  -118.251343               13
6                PlaySpan  37.396312  -121.961396               13
8         Challenge Games  30.268236   -97.740651               11
11               RocketOn  37.654660  -122.407880               10
25        Realtime Worlds  40.010492  -105.276843                9
15  Social Gaming Network  34.069849  -118.405418                9
31                Turbine  42.214483   -71.185964                7
22               Playfish  69.636216    18.931062                5
48               Altobeam  55.902143    -3.208711                3
29               WeeWorld  55.863171    -4.263517                3
28               WeeWorld  42.458900   -71.349521                1
40   Unkasoft Advergaming  41.001663    -5.693793                0
24        Realtime Worlds  56.461428    -2.968111                0

In [97]:
compdaycare.to_csv("../output/compsanddaycare.csv")